<a href="https://colab.research.google.com/github/ReccaS/Intelligent-Stock-Prediction/blob/main/Stock_Prediction_Sentiment_Pipeline_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def stock_pipeline(user_input):
    # notebooks/Project_3a.ipynb

    # Import system references
    import sys
    import os

    # Ensure project_root is in the system path
    current_dir = os.getcwd()
    project_root = os.path.abspath(os.path.join(current_dir, '..'))
    if project_root not in sys.path:
        sys.path.append(project_root)

    #print("Project root added to sys.path:", project_root in sys.path)
    #print(sys.path)

    #setup Logging
    import logging

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger(__name__)

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    #setup Logging
    import logging

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler()
        ]
    )

    logger = logging.getLogger(__name__)

    # Configure logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


    # Import necessary libraries
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns

    from datetime import datetime
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    import torch
    import torch.nn as nn
    import torch.optim as optim

    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.optimizers import Adam
    from sklearn.preprocessing import MinMaxScaler

    # Import the fetch_stock_data function (for script-based pipeline only)

    #from scripts import fetch_stock_data, transform_stock_data_to_delta, transform_with_history
    #from scripts import prepare_data_for_training, create_time_series_windows, lstm_model, prepare_sliding_window_data


 #import function dependencies
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split

    def create_time_series_windows(data, target_column, n_timesteps):
        """
        Creates sliding windows of data for time series prediction, including the target column (e.g., Close_delta)
        in the feature set but shifting the target y to predict the next Close_delta value.

        Parameters:
        - data (pd.DataFrame): The original DataFrame containing all the features and the target.
        - target_column (str): The name of the target column (e.g., 'Close_delta').
        - n_timesteps (int): The number of timesteps (sequence length) for each window.

        Returns:
        - X (np.ndarray): The array of feature windows, including Close_delta.
        - y (np.ndarray): The array of target values (Close_delta for the next time step).
        """
        #import function dependencies
        import numpy as np
        import pandas as pd
        from sklearn.model_selection import train_test_split

        X = []
        y = []

        # Loop through the data to create windows
        for i in range(len(data) - n_timesteps):
            # Include the target column in X (do not drop Close_delta)
            X_window = data.iloc[i:i + n_timesteps].values  # Include all features, including Close_delta
            y_window = data.iloc[i + n_timesteps][target_column]  # The target value is the next Close_delta

            X.append(X_window)
            y.append(y_window)

        return np.array(X), np.array(y)

    def prepare_data_for_training_with_windows(transformed_with_history_df: pd.DataFrame, target_column: str, n_timesteps: int):
        """
        Prepares the transformed DataFrame by generating time series windows and splitting them
        into training and testing sets with an 80/20 ratio.

        Parameters:
        - transformed_with_history_df (pd.DataFrame): The DataFrame containing the transformed historical data.
        - target_column (str): The name of the target column (e.g., 'Close_delta').
        - n_timesteps (int): The number of timesteps (sequence length) for each window.

        Returns:
        - X_train (np.ndarray): Training set features.
        - X_test (np.ndarray): Testing set features.
        - y_train (np.ndarray): Training set target.
        - y_test (np.ndarray): Testing set target.
        """

        # Step 1: Create time series windows
        X, y = create_time_series_windows(transformed_with_history_df, target_column, n_timesteps)

        # Step 2: Randomly split the windows into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

        return X_train, X_test, y_train, y_test

    import yfinance as yf
    import pandas as pd
    from typing import Optional
    import logging
    from datetime import datetime

    def fetch_stock_data(
        ticker: str = 'SPY',
        start_date: str = '2024-01-01',
        end_date: Optional[str] = None
    ) -> pd.DataFrame:
        """
        Fetches historical stock data for a given ticker symbol between specified dates.
        Ensures that ticker symbols are removed from column names if included.

        Parameters:
        - ticker (str, optional): The stock ticker symbol (default is 'SPY').
        - start_date (str, optional): The start date in 'YYYY-MM-DD' format (default is '2024-01-01').
        - end_date (Optional[str], optional): The end date in 'YYYY-MM-DD' format.
        If not provided, defaults to the current date.

        Returns:
        - pd.DataFrame: A DataFrame containing the stock data with simplified column names.
        """



        logger = logging.getLogger(__name__)  # Get a logger for this module

        try:
            # If end_date is not provided, set it to today's date
            if end_date is None:
                end_date = datetime.today().strftime('%Y-%m-%d')
                logger.info(f"No end_date provided. Using current date: {end_date}")
            else:
                logger.info(f"End date provided: {end_date}")

            logger.info(f"Fetching data for ticker: {ticker} from {start_date} to {end_date}")

            # Download data with ticker as a string to get single-level or multi-level columns
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)

            if data.empty:
                logger.warning(f"No data found for ticker '{ticker}' between {start_date} and {end_date}.")
                return pd.DataFrame()

            # Reset the index to make Date a column
            logger.info("Resetting index to make 'Date' a column.")
            data = data.reset_index()

            # Flatten column names if they are multi-level (i.e., contain the ticker symbol)
            if isinstance(data.columns, pd.MultiIndex):
                logger.info("Flattening multi-level column names (removing ticker symbol).")
                data.columns = data.columns.get_level_values(0)  # Keep only the first level (e.g., 'Close')

            logger.info(f"Successfully fetched and simplified data for ticker '{ticker}'.")
            return data

        except Exception as e:
            logger.error(f"Error fetching data for ticker '{ticker}': {e}")
            return pd.DataFrame()  # Return empty DataFrame on failure


    import pandas as pd

    def prepare_sliding_window_data(transformed_data_df: pd.DataFrame):
        """
        Prepares the data by dropping the specified columns and keeping only the delta values.

        Parameters:
        - transformed_data_df (pd.DataFrame): The DataFrame containing the transformed historical data.

        Returns:
        - pd.DataFrame: The DataFrame containing only the delta values.
        """
        # Columns to drop
        columns_to_drop = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']

        # Drop the specified columns and return only delta values
        delta_columns_df = transformed_data_df.drop(columns=columns_to_drop, errors='ignore')

        return delta_columns_df



    from sklearn.model_selection import train_test_split
    import pandas as pd

    def prepare_data_for_training(transformed_with_history_df: pd.DataFrame):
        """
        Prepares the transformed DataFrame by dropping the original High and Low columns (not delta ones)
        and splitting the data into training and testing sets with an 80/20 ratio.

        Returns both feature sets (X) and target sets (y).

        Parameters:
        - transformed_with_history_df (pd.DataFrame): The DataFrame containing the transformed historical data.

        Returns:
        - X_train (pd.DataFrame): Training set features.
        - X_test (pd.DataFrame): Testing set features.
        - y_train (pd.Series): Training set target.
        - y_test (pd.Series): Testing set target.
        """
        # Step 1: Drop only the original 'High' and 'Low' columns (not the delta columns)
        columns_to_drop = ['High_delta', 'Low_delta']  # Exact column names to drop
        cleaned_df = transformed_with_history_df.drop(columns=columns_to_drop, errors='ignore')

        # Step 2: Extract Close_delta as the target variable
        X = cleaned_df.drop(columns=['Close_delta'])  # Features (all columns except 'Close_delta')
        y = cleaned_df['Close_delta']  # Target variable

        # Step 3: Split the cleaned DataFrame into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        return X_train, X_test, y_train, y_test


    # scripts/transform_stock_data.py

    import pandas as pd
    from typing import Optional, List
    import logging

    # Initialize a logger for this module
    logger = logging.getLogger(__name__)

    def transform_stock_data_to_delta(
        df: pd.DataFrame,
        columns_to_exclude: Optional[List[str]] = None,
        columns_to_calculate: Optional[List[str]] = None,
        columns_to_keep: Optional[List[str]] = None
    ) -> pd.DataFrame:
        """
        Transforms the stock data by excluding specified columns, keeping certain columns
        without calculation, and calculating the proportion of increase or decrease (delta)
        from the prior row value for specified columns. Increases are positive, decreases
        are negative, and no change is zero.

        Parameters:
        - df (pd.DataFrame): The input DataFrame containing stock data.
        - columns_to_exclude (Optional[List[str]], optional): List of column names to drop from the DataFrame.
        - columns_to_calculate (Optional[List[str]], optional): List of column names to calculate deltas for.
        If None, all numeric columns except those in 'columns_to_exclude' and 'columns_to_keep' will be transformed.
        - columns_to_keep (Optional[List[str]], optional): List of column names to keep in the DataFrame but exclude from delta calculation.

        Returns:
        - pd.DataFrame: A DataFrame with the specified columns transformed to deltas, with certain columns excluded and others kept as-is.
        """
        logger.info("Starting transformation of stock data to deltas.")

        try:
            # Step 1: Drop columns to exclude
            if columns_to_exclude:
                logger.info(f"Dropping columns: {columns_to_exclude}")
                df = df.drop(columns=columns_to_exclude, errors='ignore')
            else:
                logger.info("No columns specified for exclusion.")

            # Step 2: Select columns for delta calculation
            if columns_to_calculate is None:
                # Automatically select all numeric columns except those in columns_to_keep
                numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
                columns_to_calculate = [col for col in numeric_columns if col not in (columns_to_keep or [])]
                logger.info(f"No specific columns provided for delta calculation. Automatically selected: {columns_to_calculate}")
            else:
                # Validate that specified columns exist in the DataFrame
                missing_cols = [col for col in columns_to_calculate if col not in df.columns]
                if missing_cols:
                    logger.error(f"The following specified columns for calculation are not in the DataFrame: {missing_cols}")
                    raise ValueError(f"Missing columns for calculation: {missing_cols}")
                logger.info(f"Calculating deltas for specified columns: {columns_to_calculate}")

            # Step 3: Create a copy to avoid modifying the original DataFrame
            transformed_df = df.copy()

            # Step 4: Calculate delta (proportion change) for each specified column
            for col in columns_to_calculate:
                logger.info(f"Transforming column: {col}")
                # Compute the difference from the previous row and divide by the previous row's value
                transformed_df[f"{col}_delta"] = transformed_df[col].diff() / transformed_df[col].shift(1)

                # Round the delta values for better readability (optional)
                transformed_df[f"{col}_delta"] = transformed_df[f"{col}_delta"].round(4)

            # Step 5: Drop the first row as it will contain NaN values after diff()
            logger.info("Dropping the first row with NaN values after delta calculation.")
            transformed_df = transformed_df.dropna()

            logger.info("Successfully transformed stock data to deltas.")
            return transformed_df

        except Exception as e:
            logger.error(f"An error occurred while transforming stock data: {e}")
            raise  # Re-raise the exception after logging

    def transform_with_history(transformed_df: pd.DataFrame, history_length: int = 25) -> pd.DataFrame:
        """
        Adds history columns to the transformed data by keeping only specific columns and
        appending the delta values for the past 'history_length' rows as new columns.

        Parameters:
        - transformed_df (pd.DataFrame): The transformed DataFrame containing delta columns.
        - history_length (int): The number of historical rows to append as new columns. Default is 25.

        Returns:
        - pd.DataFrame: A DataFrame with the historical delta values appended as new columns.
        """
        logger.info(f"Starting transformation with {history_length} rows of history.")

        # Step 1: Keep only the 'Date', 'Open_delta', 'High_delta', 'Low_delta', and 'Close_delta' columns
        columns_to_keep = ['Date', 'Open_delta', 'High_delta', 'Low_delta', 'Close_delta']

        # Check if the required columns are present in the DataFrame
        missing_cols = [col for col in columns_to_keep if col not in transformed_df.columns]
        if missing_cols:
            logger.error(f"Missing columns: {missing_cols} in transformed_df")
            raise ValueError(f"Required columns are missing: {missing_cols}")

        # Create a new DataFrame with the selected columns
        transformed_with_history_df = transformed_df[columns_to_keep].copy()

        # Step 2: Append historical columns using shift
        for i in range(1, history_length):
            logger.info(f"Adding historical columns for lag: {i}")

            # Shift each delta column by i rows and create new column names
            transformed_with_history_df[f'Open_delta-{i}'] = transformed_df['Open_delta'].shift(i)
            transformed_with_history_df[f'High_delta-{i}'] = transformed_df['High_delta'].shift(i)
            transformed_with_history_df[f'Low_delta-{i}'] = transformed_df['Low_delta'].shift(i)
            transformed_with_history_df[f'Close_delta-{i}'] = transformed_df['Close_delta'].shift(i)

        # Step 3: Drop rows that have NaN values due to shifting (i.e., the first 'history_length' rows)
        transformed_with_history_df = transformed_with_history_df.dropna().reset_index(drop=True)

        logger.info("Successfully transformed data with historical columns.")

        return transformed_with_history_df


    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from tensorflow.keras.optimizers import Adam

    def lstm_model(X_train, y_train, X_test, y_test, n_timesteps, n_features, num_layers, units_per_layer,
                learning_rate=0.001, epochs=20, batch_size=32):
        """
        LSTM model for time series prediction with configurable layers, learning rate, epochs, and batch size.

        Parameters:
        - X_train (np.ndarray): Training features.
        - y_train (np.ndarray): Training target.
        - X_test (np.ndarray): Testing features.
        - y_test (np.ndarray): Testing target.
        - n_timesteps (int): Number of timesteps in each sequence.
        - n_features (int): Number of features in each sequence.
        - num_layers (int): Number of LSTM layers.
        - units_per_layer (list of int): Number of units in each LSTM layer.
        - learning_rate (float, optional): Learning rate for the Adam optimizer.
        - epochs (int, optional): Number of epochs for training the model.
        - batch_size (int, optional): Batch size for model training.

        Returns:
        - y_pred (np.ndarray): Predicted target values for the test set.
        """
        assert len(units_per_layer) == num_layers, "Length of `units_per_layer` must match `num_layers`"

        # Reshape the data for LSTM (3D shape: [samples, timesteps, features])
        X_train = X_train.reshape((X_train.shape[0], n_timesteps, n_features))
        X_test = X_test.reshape((X_test.shape[0], n_timesteps, n_features))

        # Build LSTM model
        model = Sequential()

        # Add LSTM layers dynamically based on `num_layers` and `units_per_layer`
        for i in range(num_layers):
            if i == num_layers - 1:  # Last layer should not return sequences
                model.add(LSTM(units_per_layer[i], activation='relu', input_shape=(n_timesteps, n_features)))
            else:  # Intermediate layers should return sequences
                model.add(LSTM(units_per_layer[i], activation='relu', return_sequences=True))

        # Add output Dense layer
        model.add(Dense(1))

        # Compile the model with the Adam optimizer and mean squared error loss
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

        # Train the model with the provided epochs and batch size
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=2)

        # Predict and evaluate
        y_pred = model.predict(X_test)

        return y_pred, model

    # Ask the user for a ticker symbol
    ticker = user_input

    # Define start and end dates
    start_date = '2021-01-01'

    # If end_date is not provided, you can set it to today's date
    end_date = datetime.today().strftime('%Y-%m-%d')  # Default to today's date if not specified

    # Print the ticker and date range to confirm
    print(f"Fetching data for {ticker} from {start_date} to {end_date}")

    # Fetch stock data
    stock_data_df = fetch_stock_data(ticker, start_date, end_date)
    #stock_data_df = fetch_stock_data(ticker, start_date)

    # Check if data is fetched successfully
    if not stock_data_df.empty:
        # Display the first few rows
        display(stock_data_df.head())
    else:
        print("No data to display.")


    # Transform Stock Data to Deltas
    columns_to_exclude = ['Adj Close']  # Drop 'Adj Close'
    columns_to_keep = []  # Keep 'Volume' but exclude from delta calculation
    columns_to_calculate = ['Open', 'High', 'Low', 'Close', 'Volume']  # Calculate deltas for 'Open' and 'Close'

    transformed_data_df = transform_stock_data_to_delta(
        stock_data_df,
        columns_to_exclude=columns_to_exclude,
        columns_to_calculate=columns_to_calculate,
        columns_to_keep=columns_to_keep
    )

    #transformed_data_df = transform_stock_data_to_delta(stock_data_df)
    #transformed_data_df = transform_stock_data_to_delta(stock_data_df, exclude=['Volume'])

    # Display Transformed Data
    if not transformed_data_df.empty:
        display(transformed_data_df.head())
    else:
        print("No transformed data to display.")

    # Prepare the data by removing non-delta columns
    delta_only_df = prepare_sliding_window_data(transformed_data_df)

    # Display the first few rows of the resulting DataFrame
    display(delta_only_df.head())

    # Define parameters
    n_timesteps = 100  # Number of timesteps (sequence length)

    # Step 1: Create the time series windows (X and y)
    X, y = create_time_series_windows(delta_only_df, 'Close_delta', n_timesteps)

    # Display the first few occurrences of the X and y arrays
    print("First 5 entries of X:\n", X[:5])  # Display first 5 rows
    print("First 5 entries of y:\n", y[:5])  # Display first 5 target values

    # Step 2: Split the data into training and testing sets (80/20 split)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # Display the shapes of the training and testing sets
    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    # Initialize the scaler for X and y values
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    # Reshape X data to 2D for scaling, keeping the last dimension as features
    n_samples_train = X_train.shape[0]
    n_samples_test = X_test.shape[0]

    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])  # Reshape to 2D: [samples * timesteps, features]
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

    # Apply scaling to X features (fit on X_train, transform both X_train and X_test)
    X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(n_samples_train, X_train.shape[1], X_train.shape[2])
    X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(n_samples_test, X_test.shape[1], X_test.shape[2])

    # Reshape y values to 2D (required by MinMaxScaler)
    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    # Apply scaling to y values
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)


    # Define parameters
    #n_timesteps = 25
    n_features = X_train_scaled.shape[2]

    # Define the number of layers and units per layer
    num_layers = 3
    units_per_layer = [50, 100, 50]  # 3 layers with 50, 100, and 50 units, respectively

    # Call the LSTM model
    y_pred_scaled, model = lstm_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, n_timesteps, n_features, num_layers, units_per_layer,
                            learning_rate=0.00001, epochs=70, batch_size=28)
    # Output the predictions
    #print("Predictions from LSTM model:", y_pred_scaled)

    # Step 1: Inverse transform the predicted and actual y values to the original scale
    y_pred_original = scaler_y.inverse_transform(y_pred_scaled)
    y_test_original = scaler_y.inverse_transform(y_test_scaled)

    # Display the first few occurrences of prediction and test scaled and unscaled arrays
    print("First 5 entries of y_pred_scaled", y_pred_scaled[:5])  # Display first 5 rows
    print("First 5 entries of y_pred_original", y_pred_original[:5])  # Display first 5 rows
    print("First 5 entries of y_test_scaled", y_test_scaled[:5])  # Display first 5 target values
    print("First 5 entries of y_test_original", y_test_original[:5])  # Display first 5 target values

    # Mean Squared Error
    mse = mean_squared_error(y_test_original, y_pred_original)
    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    # Mean Absolute Error
    mae = mean_absolute_error(y_test_original, y_pred_original)
    # R-squared
    r2 = r2_score(y_test_original, y_pred_original)

    # Print the metrics
    print(f"Mean Squared Error (MSE): {mse:.6f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
    print(f"Mean Absolute Error (MAE): {mae:.6f}")
    print(f"R-squared (R²): {r2:.6f}")


    # Create a DataFrame to compare actual and predicted values
    comparison_df = pd.DataFrame({'Actual': y_test_original.flatten(), 'Predicted': y_pred_original.flatten()})
    # Calculate the difference (error)
    comparison_df['Difference'] = comparison_df['Actual'] - comparison_df['Predicted']
    #print(comparison_df.head(25))


    # Get the sign of the actual and predicted values
    actual_sign = np.sign(comparison_df['Actual'])
    predicted_sign = np.sign(comparison_df['Predicted'])
    #print(actual_sign.head(25))
    #print(predicted_sign.head(25))
    # Check where the signs match
    sign_matches = actual_sign == predicted_sign
    print(sign_matches.head(100))

    # Calculate the percentage of sign matches
    percentage_match = sign_matches.mean() * 100

    print(f"Percentage of Sign Matches: {percentage_match :.2f}%")


    # Get the prediction for tomorrow's Close_delta

    """
    Predict the next Close_delta based on the most recent n_timesteps of data.

    Using:
    - model: The trained LSTM model.
    - delta_only_df (pd.DataFrame): The DataFrame containing all historical data, including the latest Close_delta.
    - n_timesteps (int): The number of timesteps (sequence length) used in the model.
    - scaler_X: The scaler used to scale the features (X).
    - scaler_y: The scaler used to scale the target (y).

    Returns:
    - float: The predicted Close_delta for the next day.
    """
    # Step 1: Extract the last n_timesteps rows from the data (to be used as input for prediction)
    last_window = delta_only_df[-n_timesteps:].values.reshape(1, n_timesteps, delta_only_df.shape[1])

    # Step 2: Scale the input features
    last_window_scaled = scaler_X.transform(last_window.reshape(-1, last_window.shape[-1])).reshape(1, n_timesteps, -1)

    # Step 3: Use the model to predict the next Close_delta (scaled)
    predicted_close_delta_scaled = model.predict(last_window_scaled)

    # Step 4: Inverse transform the prediction to get the original scale of Close_delta
    predicted_close_delta = scaler_y.inverse_transform(predicted_close_delta_scaled)

    predicted_close_delta = predicted_close_delta[0][0]  # Return the predicted value

    # Decision logic: Buy if positive, Sell if negative
    result = (
                f"Predicted Close increase for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: BUY with {percentage_match :.2f}% confidence"
                if predicted_close_delta > 0
                else f"Predicted Close decrease for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: SELL with {percentage_match :.2f}% confidence")

    return result

In [ ]:
#stock_pipeline('AAPL')

**Sentiment Analysis Pipeline**

In [ ]:
from transformers import pipeline
import yfinance as yf
import numpy as np
import tensorflow as tf



In [ ]:
# Initialize the sentiment analysis pipeline from Hugging Face
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_analysis = pipeline("sentiment-analysis",model=model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Function to fetch recent stock-related news using yfinance
def fetch_stock_news(stock_symbol, num_articles=5):
    stock = yf.Ticker(stock_symbol)
    news = stock.news  # Get stock news

    if not news:
        return []  # Return an empty list if no news is available



    #Change num_articles to int for Gradio use

    num_articles = int(num_articles)
    # Extract top `num_articles` news articles
    news_headlines = [article['title'] for article in news[:num_articles]]
    return news_headlines

In [ ]:
# Perform sentiment analysis on stock news headlines
def analyze_sentiment_on_news(stock_symbol, num_articles):
    # Fetch stock news
    headlines = fetch_stock_news(stock_symbol, num_articles)

    # Check if headlines are available
    if not headlines:
        print(f"No news found for {stock_symbol}")
        return

    #print(f"News headlines for {stock_symbol}:\n")

    # Perform sentiment analysis on each headline
    results = sentiment_analysis(headlines)

    # Initialize lists to store sentiment and scores
    sentiments = []
    scores = []

    for result in results:
        score = result['score']
        scores.append(score)
        sentiment = result['label']
        if sentiment == 'POSITIVE':
            sentiments.append(1)
        elif sentiment == 'NEGATIVE':
            sentiments.append(-1)
        else:
            sentiments.append(0)



    # Calculate the average sentiment score
    average_sentiment = np.mean(sentiments)

    # Determine overall sentiment
    if average_sentiment > 0:
      overall_sentiment = "Positive"
    elif average_sentiment < 0:
      overall_sentiment = "Negative"
    else:
      overall_sentiment = "Neutral"

      # Create a function that averages scores
    def average(numbers):
     sum_of_numbers = sum(numbers)
     average = (sum_of_numbers / len(numbers)) * 100
     return average

    #Run average function
    score_average = average(scores)


    #Return the average score and overall sentiment for the stock ticker
    return f"{score_average:.2f}% | {overall_sentiment}"




In [ ]:
 # Example: Analyze sentiment on Apple (AAPL) stock news
stock_symbol = "AAPL"
num_articles = 5
analyze_sentiment_on_news(stock_symbol, num_articles)

'99.15% | Positive'

**Combined Pipelines**

In [ ]:
def combined_pipelines(ticker, num_articles=None):
  prediction_one = stock_pipeline(ticker)
  if num_articles:
    prediction_two = analyze_sentiment_on_news(ticker,num_articles)
  else:
    prediction_two = analyze_sentiment_on_news(ticker,5)
  return prediction_one, prediction_two

**Gradio Interface**

In [ ]:
#If gradio not installed, unhash install gradio below and rerun code

#!pip install gradio
import gradio as gr

input_component_one = gr.Textbox(label="Stock Ticker")
input_component_two = gr.Textbox(label="Number of articles")
output_component_one = gr.Textbox(label="Stock Price Prediction")
output_component_two = gr.Textbox(label="Public Sentiment Score ")

output_component = [output_component_one, output_component_two]

interface = gr.Interface(fn= combined_pipelines, inputs=[input_component_one,input_component_two], outputs=[output_component_one, output_component_two])
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4b2e67a3fc1c4be92a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Fetching data for AAPL from 2021-01-01 to 2024-10-31


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-01-04 00:00:00+00:00,126.683434,129.410004,133.610001,126.760002,133.520004,143301900
1,2021-01-05 00:00:00+00:00,128.249756,131.009995,131.740005,128.429993,128.889999,97664900
2,2021-01-06 00:00:00+00:00,123.932648,126.599998,131.050003,126.379997,127.720001,155088000
3,2021-01-07 00:00:00+00:00,128.161652,130.919998,131.630005,127.860001,128.360001,109578200
4,2021-01-08 00:00:00+00:00,129.267822,132.050003,132.630005,130.229996,132.429993,105158200


Price,Date,Close,High,Low,Open,Volume,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,2021-01-05 00:00:00+00:00,131.009995,131.740005,128.429993,128.889999,97664900,-0.0347,-0.0140,0.0132,0.0124,-0.3185
2,2021-01-06 00:00:00+00:00,126.599998,131.050003,126.379997,127.720001,155088000,-0.0091,-0.0052,-0.0160,-0.0337,0.5880
3,2021-01-07 00:00:00+00:00,130.919998,131.630005,127.860001,128.360001,109578200,0.0050,0.0044,0.0117,0.0341,-0.2934
4,2021-01-08 00:00:00+00:00,132.050003,132.630005,130.229996,132.429993,105158200,0.0317,0.0076,0.0185,0.0086,-0.0403
5,2021-01-11 00:00:00+00:00,128.979996,130.169998,128.500000,129.190002,100384500,-0.0245,-0.0185,-0.0133,-0.0232,-0.0454


Price,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,-0.0347,-0.0140,0.0132,0.0124,-0.3185
2,-0.0091,-0.0052,-0.0160,-0.0337,0.5880
3,0.0050,0.0044,0.0117,0.0341,-0.2934
4,0.0317,0.0076,0.0185,0.0086,-0.0403
5,-0.0245,-0.0185,-0.0133,-0.0232,-0.0454


First 5 entries of X:
 [[[-3.470e-02 -1.400e-02  1.320e-02  1.240e-02 -3.185e-01]
  [-9.100e-03 -5.200e-03 -1.600e-02 -3.370e-02  5.880e-01]
  [ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  ...
  [ 1.440e-02  3.000e-03  3.000e-03 -1.600e-03  1.413e-01]
  [-6.700e-03 -7.200e-03  8.000e-04 -4.000e-04 -2.143e-01]
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]]

 [[-9.100e-03 -5.200e-03 -1.600e-02 -3.370e-02  5.880e-01]
  [ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  [ 3.170e-02  7.600e-03  1.850e-02  8.600e-03 -4.030e-02]
  ...
  [-6.700e-03 -7.200e-03  8.000e-04 -4.000e-04 -2.143e-01]
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]
  [-6.900e-03 -1.440e-02 -4.200e-03 -5.300e-03 -2.464e-01]]

 [[ 5.000e-03  4.400e-03  1.170e-02  3.410e-02 -2.934e-01]
  [ 3.170e-02  7.600e-03  1.850e-02  8.600e-03 -4.030e-02]
  [-2.450e-02 -1.850e-02 -1.330e-02 -2.320e-02 -4.540e-02]
  ...
  [-4.100e-03  2.000e-03 -1.060e-02 -1.240e-02  6.725e-01]
  [-6.900e-

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 - 11s - 426ms/step - loss: 0.1933 - val_loss: 0.1841
Epoch 2/70
25/25 - 8s - 323ms/step - loss: 0.1862 - val_loss: 0.1770
Epoch 3/70
25/25 - 5s - 216ms/step - loss: 0.1793 - val_loss: 0.1704
Epoch 4/70
25/25 - 8s - 337ms/step - loss: 0.1727 - val_loss: 0.1640
Epoch 5/70
25/25 - 6s - 250ms/step - loss: 0.1663 - val_loss: 0.1576
Epoch 6/70
25/25 - 4s - 144ms/step - loss: 0.1598 - val_loss: 0.1512
Epoch 7/70
25/25 - 5s - 218ms/step - loss: 0.1536 - val_loss: 0.1453
Epoch 8/70
25/25 - 6s - 256ms/step - loss: 0.1476 - val_loss: 0.1391
Epoch 9/70
25/25 - 10s - 418ms/step - loss: 0.1413 - val_loss: 0.1329
Epoch 10/70
25/25 - 8s - 332ms/step - loss: 0.1349 - val_loss: 0.1263
Epoch 11/70
25/25 - 5s - 200ms/step - loss: 0.1280 - val_loss: 0.1193
Epoch 12/70
25/25 - 4s - 155ms/step - loss: 0.1208 - val_loss: 0.1120
Epoch 13/70
25/25 - 5s - 193ms/step - loss: 0.1136 - val_loss: 0.1053
Epoch 14/70
25/25 - 8s - 318ms/step - loss: 0.1068 - val_loss: 0.0981
Epoch 15/70
25/25 - 7s - 295ms/step - 